In [ ]:
#hide
from utilities.ipynb_docgen import *
from nbdev import *

!date

Fri Feb 26 15:15:52 PST 2021


# B1259 Analysis
> Create a B1259-63 (aka PSR J1302-6350) light curve using Bayesian Blocks  

* Ref from [HESS](https://arxiv.org/pdf/astro-ph/0506280.pdf)

* [Some results from *fermi*](https://arxiv.org/pdf/1912.05868.pdf)

In [ ]:
#collapse_hide
from light_curves.tools import *
from wtlike.config import *
from utilities.ipynb_docgen import *
from nbdev import *

bb_lc = None
lc = None

def B1259(name='PSR_B1259-63', expect=1):
    """
    ## Fit to all data
    
    Combine the likelihoods for all the data, check the fit.
    
    {outp}
    This plot is a consistency check of the weighted likelihood for the full interval. 
    
    {fig1}
    
    ## The full daily-interval light curve, showing the BB partitions
    {fig2}
    
    ### Periastron dates
    Assuming {p}-day orbital period, the MJD and UTC values are:
    
    {utc}
    
    Expand the above about those dates
    
    {fig3}
    
    ### Table with the intervals:
    {lc_display}
    
    Download it: [{lc_file}]({lc_file}).
    """
    global lc, bb_lc
    config = Config()
    assert config.valid, f"Invalid configureation {config}"
    
    plt.rc('font', size=14)
    source = PointSource(name)
    
    with capture_print('Output from setup') as outp:
        ll = all_data_likelihood(config, source)

    #print(ll,'\nFit: ', ll.fit_info())
    fig1, ax1 = plt.subplots(num=1, figsize=(3,2))
    ll.plot(xlim =( 0.5, 1.5), ax=ax1)
   
    with capture_print('Analysis output') as outp:
        lc, bb_lc = analyze_data(config, source)
        
    pd.set_option('display.precision', 3)#, 'display.colheader_justify','left')
    
    df = fit_table(bb_lc, expect=expect)
    df_text = monospace(str(df), 'BB fit table', open=True)
        
    plt.rc('font', size=16)
    fig2, ax2 = plt.subplots(1,1, sharex=True, figsize=(20,4), num=2)
    bb_overplot(config, lc, bb_lc, ax = ax2)
    ax2.text(0.05, 0.85, name,  transform=ax2.transAxes);
    ax2.set(yscale='log')
    fig2.width=600
    
    tp, p =55546, 1237
    t = [ dict(MET=tp+n*p, UTC=UTC(tp+n*p)) for n in range(4) ]
    utc = pd.DataFrame(t) 
    
    fig3, axx = plt.subplots(3,1, sharex=False, sharey=True, figsize=(12,12), num=3)
    plt.subplots_adjust(hspace=0.3)
    for i, ax in enumerate(axx.flatten()):
        bb_overplot(config, lc, bb_lc, ax=ax, tzero=tp+i*p, xlim=(-60,150), yscale='log') #, colors=('black','wheat','blue'))
        ax.text(0.02, 0.9, UTC(tp+i*p)[:4], transform=ax.transAxes)
    fig3.width=600
    
    ### Save the BB intervals
    lc_out = bb_lc.copy()

    lc_out.loc[:,'flux'] = bb_lc.fit.apply(lambda fit: round(fit.flux,3))
    errors = bb_lc.fit.apply(lambda fit: np.array(fit.errors).round(3)).values
    elow, ehigh = [[x[i] for x in errors] for i in range(2)]
    lc_out['low']=elow
    lc_out['high']=ehigh
    lc_display = lc_out['t tw n flux low high'.split()]
    lc_file = 'images/B1259_lc.csv'
    lc_display.to_csv(lc_file)
    return locals()
nbdoc(B1259)

## Fit to all data

Combine the likelihoods for all the data, check the fit.

<details  class="nbdoc-description" >  <summary> Analysis output </summary>  <div style="margin-left: 5%"><pre>Light curve for PSR_B1259-63: Restoring from cache with key "lightfcurve_PSR_B1259-63"<br>Cell data for PSR_B1259-63: Restoring from cache with key "cells_PSR_B1259-63"<br>BB-PSR_B1259-63: Restoring from cache with key "BB-PSR_B1259-63"<br>Partitioned 3840 cells into 22 blocks, using LikelihoodFitness <br>Loaded 22 / 22 cells with exposure &gt; 0.3 for fitting<br></pre></div> </details>
This plot is a consistency check of the weighted likelihood for the full interval. 

<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 1">  <a href="images/B1259_fig_01.png" title="images/B1259_fig_01.png">    <img src="images/B1259_fig_01.png" alt="Figure 1 at images/B1259_fig_01.png" >   </a> </figure>
</div>


### The full daily-interval light curve, showing the BB partitions
<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 2">  <a href="images/B1259_fig_02.png" title="images/B1259_fig_02.png">    <img src="images/B1259_fig_02.png" alt="Figure 2 at images/B1259_fig_02.png" width=600>   </a> </figure>
</div>

### Periastron dates
Assuming 1237-day orbital period, the MJD and UTC values are:

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>MET</th>
      <th>UTC</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>55546</td>
      <td>2010-12-16 00:00</td>
    </tr>
    <tr>
      <td>56783</td>
      <td>2014-05-06 00:00</td>
    </tr>
    <tr>
      <td>58020</td>
      <td>2017-09-24 00:00</td>
    </tr>
    <tr>
      <td>59257</td>
      <td>2021-02-12 00:00</td>
    </tr>
  </tbody>
</table>
</div>

Expand the above about those dates

<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 3">  <a href="images/B1259_fig_03.png" title="images/B1259_fig_03.png">    <img src="images/B1259_fig_03.png" alt="Figure 3 at images/B1259_fig_03.png" width=600>   </a> </figure>
</div>


### Table with the intervals:
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>t</th>
      <th>tw</th>
      <th>n</th>
      <th>flux</th>
      <th>low</th>
      <th>high</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>55088.500</td>
      <td>811.000</td>
      <td>421805</td>
      <td>0.000</td>
      <td>0.000</td>
      <td>0.076</td>
    </tr>
    <tr>
      <td>55534.000</td>
      <td>80.000</td>
      <td>53111</td>
      <td>4.478</td>
      <td>3.194</td>
      <td>5.788</td>
    </tr>
    <tr>
      <td>55587.500</td>
      <td>27.000</td>
      <td>14780</td>
      <td>47.731</td>
      <td>44.203</td>
      <td>51.278</td>
    </tr>
    <tr>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
    <tr>
      <td>58089.000</td>
      <td>2.000</td>
      <td>1045</td>
      <td>117.303</td>
      <td>100.185</td>
      <td>134.777</td>
    </tr>
    <tr>
      <td>58359.000</td>
      <td>538.000</td>
      <td>309771</td>
      <td>0.000</td>
      <td>0.000</td>
      <td>0.179</td>
    </tr>
    <tr>
      <td>58662.500</td>
      <td>69.000</td>
      <td>36122</td>
      <td>0.000</td>
      <td>0.000</td>
      <td>1.460</td>
    </tr>
  </tbody>
</table>
</div>

Download it: [images/B1259_lc.csv](images/B1259_lc.csv).
